In [ ]:
import rasterio
import os
from datetime import datetime
import numpy as np

In [ ]:
# local libraries
from coupling_PCR_FM.model_functions_v2 import PCR_model, CMF_model, DFM_model
from coupling_PCR_FM.utils import config_to_dict
%config Application.log_level="INFO"

In [ ]:
# IMPORT MODEL SETTINGS FROM INI-FILE
argv1 = r'default_rename.set'
argv2 = r'paths.env'

# parse set/ini-file with central/general settings for coupling framework
config = config_to_dict(argv1)
# parse env-file for user-specific paths and environmental variables
envs = config_to_dict(argv2)
# combine
config.update(envs)
options = config
# parse dates
start_date = datetime.strptime(options['numerical_settings']['startTime'], '%Y-%m-%d')
end_date = datetime.strptime(options['numerical_settings']['endTime'], '%Y-%m-%d')

In [ ]:
# setup output dir
cwd = os.getcwd() # note: this get changed by pcr initialization later on
out_dir = options['PCRpaths']['outputDirectoryPCR']
if not os.path.isdir(out_dir):
    os.mkdir(out_dir)
print(out_dir)
print(cwd)

## create CMF model BMI object

In [ ]:
CMF_engine = os.path.join(cwd, options['CMF_engine']['CMF_path'])
print(CMF_engine)
CMF_model_dir = os.path.join(cwd, options['routing_model']['model_dir'])
CMF_config_fn = os.path.join(CMF_model_dir, options['routing_model']['model_file'])
CMF_out_dir = os.path.join(out_dir, 'CMF')
     
CMF_bmi = CMF_model(CMF_engine, CMF_config_fn, CMF_model_dir, CMF_out_dir,
                         start_date, end_date, dt=86400)


## create DFM model BMI object

In [ ]:
DFM_engine = os.path.join(cwd, options['DFM_engine']['DFM_path'])
DFM_model_dir = os.path.join(cwd, options['hydrodynamic_model']['model_dir'])
DFM_config_fn = os.path.join(DFM_model_dir, options['hydrodynamic_model']['model_file'])
DFM_out_dir = os.path.join(out_dir, 'DFM')
print(DFM_engine)
print(DFM_model_dir)
print(DFM_config_fn)
print(DFM_out_dir)


DFM_bmi = DFM_model(DFM_engine, DFM_config_fn, DFM_model_dir, DFM_out_dir,
                              start_date, end_date, dt=86400.)

## intitialize DFM model

In [ ]:
# initialize DFM first to expose model coordinates via BMI
DFM_bmi.initialize()

## couple grids based on 1d DFM indices

Couple external 1d coordinates to internal model 2d grid. 

The model **ldd grid is also deactivated** at coupled cells.

A dictionary with for each 1d xy coordinates, the index of the 2d grid (1 to 1) and its inversed dictionary (1 to n).

**Note** that this should be excecuted before the CMF model is initalized as the lddl grid modification is done offline!

In [ ]:
# get coupled dictionaries
CMF_bmi.couple_grid_to_1d(DFM_bmi)

## initialize CMF model

In [ ]:
# initialize CMF model after nextxy had been adapted
CMF_bmi.initialize()

## define exchange of values in update states function

In [ ]:
# determine area fraction for volume distribution
DFMidx = DFM_bmi.coupled_idx
CMFidx = CMF_bmi.coupled_idx
area = DFM_bmi.get_var('ba')[DFMidx]

def update_states():
    discharge_cmf = CMF_bmi.get_var('outflw')[CMFidx] * CMF_bmi.options['dt'] #[m3/dt]
    delta_water_depth_dfm = discharge_cmf * CMF_bmi.coupled_area_frac / area # [m/dt]
    DFM_bmi.set_var_index('rain', DFMidx, delta_water_depth_dfm)
        

## run coupled CMF - DFM model

In [ ]:
# run coupled model for 2 timesteps
for i in range(1):
    CMF_bmi.update()
    update_states()
#     DFM_bmi.update()


## Finalize models

In [ ]:
# PCR_bmi.finalize()
# CMF_bmi.finalize()
# DFM_bmi.finalize()
